# Import Dependencies

In [1]:
from datetime import date, timedelta, datetime
from time import sleep
import requests
import pandas as pd
# import pprint as pp
import sqlalchemy
from sqlalchemy.orm import Session
from psql_connect import get_engine, get_base
from api_keys import nyt_apikey_1, nyt_apikey_2

# Connect to SQL Database

In [2]:
# Create SQLAlchemy engine and automap base
engine = get_engine()
Base = get_base(engine)

# Create session
session = Session(engine)

# Confirm successful connection
Base.classes.keys()

['imprints', 'ownership', 'publishers', 'authors', 'books', 'lists', 'results']

In [3]:
# Assign table classes to variables
Authors = Base.classes.authors
Books = Base.classes.books
Imprints = Base.classes.imprints
Lists = Base.classes.lists
Results = Base.classes.results
Publishers = Base.classes.publishers
Ownership = Base.classes.ownership

# Get NYT Data

In [4]:
def get_NYT_Data(search_date):
    # Create URL string for API calls
    url = f"https://api.nytimes.com/svc/books/v3/lists/full-overview.json?published_date={search_date.strftime('%Y-%m-%d')}&api-key={nyt_apikey_1}"

    # Make API request
    response = requests.get(url).json()
    
    # Empty list which will later be used to create Pandas Dataframe
    book_data = []

    # Published date
    pub_date = datetime.strptime(response['results']['published_date'], '%Y-%m-%d').date()

    # Loop through each list in the response
    for blist in response['results']['lists']:

        list_name = blist['list_name']

    #     Loop through each book in the list
        for book in blist['books']:

            # Append book-related information
            book_data.append({
                'NYT List': list_name,
                'NYT List Published Date': pub_date,            
                'Title': book['title'],
                'Author': book['author'],
                'Imprint': book['publisher'],
                'Rank': book['rank'],
                'Weeks on NYT List': book['weeks_on_list'],
                'Image': book['book_image'],
                'Description': book['description']
            })
    book_df = pd.DataFrame(book_data)
    return book_df

# Update SQL Functions

In [30]:
def update_lists_table(book_df):
    print("Updating lists table...")
    # Lists in the current DataFrame
    lists_from_df = book_df['NYT List'].unique()
    # Lists in the SQL database
    lists_from_pg = list(blist[0] for blist in session.query(Lists.list_name).all())
    # Lists that are in the current DataFrame but not in the SQL database
    lists_to_add = list(blist for blist in lists_from_df if blist not in lists_from_pg)
    
    # If there are any lists in the current DataFrame but not in the SQL database, add the lists to the SQL database
    if len(lists_to_add) > 0:
        for blist in lists_to_add:
            session.add(Lists(list_name=blist))
        session.commit()

In [31]:
def update_imprints_table(book_df):
    print("Updating imprints table...")
    imprints_from_df = book_df['Imprint'].unique()
    imprints_from_pg = list(imprint[0] for imprint in session.query(Imprints.imprint_name).all())
    imprints_to_add = list(imprint for imprint in imprints_from_df if imprint not in imprints_from_pg)

    if len(imprints_to_add) > 0:
        for imprint in imprints_to_add:
            session.add(Imprints(imprint_name=imprint))
        session.commit()

In [32]:
def update_authors_table(book_df):
    print("Updating authors table...")
    authors_from_df = book_df['Author'].unique()
    authors_from_pg = list(author[0] for author in session.query(Authors.author_name).all())
    authors_to_add = list(author for author in authors_from_df if author not in authors_from_pg)

    if len(authors_to_add) > 0:
        for author in authors_to_add:
            session.add(Authors(author_name=author))
        session.commit()

In [33]:
def update_books_table(book_df):
    print("Updating books table...")
    books_from_df = book_df['Title'].unique()
    books_from_pg = list(book[0] for book in session.query(Books.book_title).all())
    
    if len(list(book for book in books_from_df if book not in books_from_pg)) > 0:
        for i in range(len(book_df)):
            title = book_df.loc[i, 'Title']

            if title not in books_from_pg:
                image = book_df.loc[i, 'Image']
                desc = book_df.loc[i, 'Description']
                author = book_df.loc[i, 'Author']
                imprint = book_df.loc[i, 'Imprint']

                author_id = session.query(Authors.author_id).filter(Authors.author_name == author).first()[0]
                imprint_id = session.query(Imprints.imprint_id).filter(Imprints.imprint_name == imprint).first()[0]

                session.add(Books(book_title = title, \
                                     book_image = image, \
                                     book_description = desc, \
                                     author_id = author_id, \
                                     imprint_id = imprint_id))

        session.commit()

In [34]:
def update_results_table(book_df):
    print("Updating results table...")
    dates_from_df = book_df['NYT List Published Date'].unique()
    dates_from_pg = list(pdate[0] for pdate in session.query(Results.published_date).group_by(Results.published_date).all())
    
    if len(list(pdate for pdate in dates_from_df if pdate not in dates_from_pg)) > 0:
        for i in range(len(book_df)):
            blist = book_df.loc[i, 'NYT List']
            title = book_df.loc[i, 'Title']
            pub_date = book_df.loc[i, 'NYT List Published Date']
            rank = int(book_df.loc[i, 'Rank'])
            weeks_on_list = int(book_df.loc[i, 'Weeks on NYT List'])

            list_id = session.query(Lists.list_id).filter(Lists.list_name == blist).first()[0]
            book_id = session.query(Books.book_id).filter(Books.book_title == title).first()[0]

            session.add(Results(list_id = list_id, \
                                   book_id = book_id, \
                                   published_date = pub_date, \
                                   rank = rank, \
                                   weeks_on_list = weeks_on_list))

        session.commit()

In [10]:
def update_database(book_df):
    update_lists_table(book_df)
    update_imprints_table(book_df)
    update_authors_table(book_df)
    update_books_table(book_df)
    update_results_table(book_df)

# Retrieve NYT Data and Update Database

In [50]:
# Loop through years
for j in reversed(range(2018, 2024)):
    # Loop through months
    for i in reversed(range(1, 13)):
        # Conditional to exclude dates after May 2023
        if not (j == 2023 and i > 5):
            search_date = date(j, i, 15)
            print(f"Beginning data retrieval for {search_date}")
            book_df = get_NYT_Data(search_date)
            print("Beginning database update")
            update_database(book_df)
            print(f"Update complete with {search_date} data")
            sleep(30)

Beginning data retrieval for 2018-12-15
Beginning database update
Updating lists table...
Updating imprints table...
Updating authors table...
Updating books table...
Updating results table...
Update complete with 2018-12-15 data
Beginning data retrieval for 2018-11-15
Beginning database update
Updating lists table...
Updating imprints table...
Updating authors table...
Updating books table...
Updating results table...
Update complete with 2018-11-15 data
Beginning data retrieval for 2018-10-15
Beginning database update
Updating lists table...
Updating imprints table...
Updating authors table...
Updating books table...
Updating results table...
Update complete with 2018-10-15 data
Beginning data retrieval for 2018-09-15
Beginning database update
Updating lists table...
Updating imprints table...
Updating authors table...
Updating books table...
Updating results table...
Update complete with 2018-09-15 data
Beginning data retrieval for 2018-08-15
Beginning database update
Updating lists

# Close Session

In [55]:
session.close()